In [35]:
import pandas as pd
import csv, sqlite3

In [36]:
df = pd.read_csv("../cleaning-data/web-cleaned-data.csv")
df.head()

,FlightNo.,Dateandtime(UTC),Launchsite,Payloadmass,Orbit,Customer,Launchoutcome,Boosterlanding,BoosterVersion,Serial
0,1,2010-06-04,CCSFS SLC-40,8933.290816,LEO,SpaceX,Success,Failure(parachute),F9 v1.0,B0003
1,2,2010-12-08,CCSFS SLC-40,8933.290816,LEO,NASA(COTS),Success,Failure(parachute),F9 v1.0,B0004
2,3,2012-05-22,CCSFS SLC-40,525.000000,LEO,NASA(COTS),Success,No attempt,F9 v1.0,B0005
3,4,2012-10-08,CCSFS SLC-40,4700.000000,LEO,NASA(CRS),Success,No attempt,F9 v1.0,B0006
4,5,2013-03-01,CCSFS SLC-40,4877.000000,LEO,NASA(CRS),Success,No attempt,F9 v1.0,B0007


In [37]:
conn = sqlite3.connect('SpaceX.db')
df.to_sql("SPACEXLAUNCH", conn, if_exists="replace", index=False)
conn.commit()
conn.close()

In [38]:
%load_ext sql
conn = sqlite3.connect('SpaceX.db')
cur = conn.cursor()
%sql sqlite:///SpaceX.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


1. Display number of unique launch site

In [39]:
%%sql 
select Launchsite, count("FlightNo.") as LaunchTotal
from spacexlaunch
group by Launchsite

 * sqlite:///SpaceX.db
Done.


Launchsite,LaunchTotal
CCSFS SLC-40,115
KSC LC-39A,57
VSFB SLC-4E,36


2. Display total payload mass, average per launch carried by boosters launched by NASA

In [48]:
%%sql 
select customer, sum(payloadmass) as TotalPayloadMass, count(customer) as LaunchTotal, avg(payloadmass) as AveragePerLaunch
from spacexlaunch
where customer like '%NASA%'
group by customer
order by TotalPayloadMass desc

 * sqlite:///SpaceX.db
Done.


Customer,TotalPayloadMass,LaunchTotal,AveragePerLaunch
NASA(CTS),77050.0,6,12841.666666666666
NASA(CRS),71326.0,26,2743.3076923076924
NASA(CCDev),12530.0,1,12530.0
NASA(CCP),12500.0,1,12500.0
NASA(CCD),12055.0,1,12055.0
NASA(COTS),9458.290816326531,2,4729.1454081632655
Iridium Communications/GFZ/NASA,6460.0,1,6460.0
NASA/CNES,2200.0,1,2200.0
NASA(LSP),1311.0,3,437.0
NASA/NOAA/ESA/EUMETSAT,1192.0,1,1192.0


3. How many booster does SpaceX have? Which booster has the highest average payload?

In [55]:
%%sql 
select boosterversion, sum(payloadmass) as TotalPayloadMass, avg(payloadmass) as AveragePayloadMass, count(boosterversion) as LaunchTotal
from spacexlaunch
group by boosterversion
order by AveragePayloadMass desc

 * sqlite:///SpaceX.db
Done.


BoosterVersion,TotalPayloadMass,AveragePayloadMass,LaunchTotal
F9 B5,1615075.3265306123,10625.495569280343,152
F9 v1.0,27968.581632653062,5593.716326530613,5
F9 B4,63575.29081632653,5297.940901360545,12
F9 FT,113485.29081632654,4728.553784013606,24
F9 v1.1,38020.0,2534.6666666666665,15


4. How many type of booster landing? List the first date of success ground pad?

In [62]:
%%sql
select boosterlanding, min("Dateandtime(UTC)") as FirstSuccessDate
from spacexlaunch
where boosterlanding = "Success(ground pad)"

 * sqlite:///SpaceX.db
Done.


Boosterlanding,FirstSuccessDate
Success(ground pad),2015-12-22


5. Which type of booster landing has highest success rate?

* A controlled ocean landing means the ocean test controlled descent, for the sole purpose of gathering test data, such boosters were destroyed at sea. Controlled means success test, soft vertical ocean touchdown. Uncontrolled means failure test, ocean touchdown control failed.

* Drone ship: land on a drone ship at sea

* Ground pad: land on a ground pad near the launch site

* Failure parachute: attempt to recover the first stage by parachuting it into the ocean but fail

In [73]:
%%sql
select boosterlanding, count(boosterlanding) as LaunchTotal
from spacexlaunch
group by boosterlanding

 * sqlite:///SpaceX.db
Done.


Boosterlanding,LaunchTotal
Controlled(ocean),5
Failure(drone ship),8
Failure(ground pad),1
Failure(parachute),2
No attempt,24
Precluded(drone ship),1
Success(drone ship),136
Success(ground pad),29
Uncontrolled(ocean),2


In [74]:
%%sql
select count(Boosterlanding) as TotalSuccess
from spacexlaunch
where boosterlanding in ('Controlled(ocean)', 'Success(drone ship)', 'Success(ground pad)')

 * sqlite:///SpaceX.db
Done.


TotalSuccess
170


In [95]:
%%sql
select count(Boosterlanding) as TotalFailure
from spacexlaunch
where boosterlanding in ('Failure(drone ship)', 'Failure(ground pad)', 'Failure(parachute)', 'Uncontrolled(ocean)')

 * sqlite:///SpaceX.db
Done.


TotalFailure
13


In [92]:
%%sql
select boosterlanding, boosterversion, orbit, count(Boosterlanding) as TotalOcean
from spacexlaunch
where boosterlanding like '%ocean%'
group by boosterlanding, boosterversion, orbit

 * sqlite:///SpaceX.db
Done.


Boosterlanding,BoosterVersion,Orbit,TotalOcean
Controlled(ocean),F9 FT,GTO,1
Controlled(ocean),F9 FT,PO,1
Controlled(ocean),F9 v1.1,ES-L1,1
Controlled(ocean),F9 v1.1,LEO,2
Uncontrolled(ocean),F9 v1.1,LEO,1
Uncontrolled(ocean),F9 v1.1,PO,1


In [93]:
%%sql
select boosterlanding, boosterversion, orbit, count(Boosterlanding) as TotalGroundPad
from spacexlaunch
where boosterlanding like "%ground%"
group by boosterlanding, boosterversion, orbit

 * sqlite:///SpaceX.db
Done.


Boosterlanding,BoosterVersion,Orbit,TotalGroundPad
Failure(ground pad),F9 B5,LEO,1
Success(ground pad),F9 B4,LEO,3
Success(ground pad),F9 B5,BLT,1
Success(ground pad),F9 B5,LEO,7
Success(ground pad),F9 B5,PO,2
Success(ground pad),F9 B5,SSO,10
Success(ground pad),F9 FT,LEO,6


In [98]:
%%sql
select boosterlanding, boosterversion, orbit, count(Boosterlanding) as TotalDroneShip
from spacexlaunch
where boosterlanding like "%drone%"
group by boosterlanding, boosterversion, orbit

 * sqlite:///SpaceX.db
Done.


Boosterlanding,BoosterVersion,Orbit,TotalDroneShip
Failure(drone ship),F9 B5,LEO,3
Failure(drone ship),F9 FT,GTO,2
Failure(drone ship),F9 v1.1,LEO,3
Precluded(drone ship),F9 v1.1,LEO,1
Success(drone ship),F9 B4,GTO,1
Success(drone ship),F9 B4,HEO,1
Success(drone ship),F9 B4,PO,1
Success(drone ship),F9 B5,BLT,1
Success(drone ship),F9 B5,GTO,19
Success(drone ship),F9 B5,Heliocentric,1


6. List the name of booster which have success in droneship

In [66]:
%%sql
select boosterversion, count(boosterversion) as TotalSuccess
from spacexlaunch
where boosterlanding = "Success(drone ship)"
group by boosterversion
order by TotalSuccess desc

 * sqlite:///SpaceX.db
Done.


BoosterVersion,TotalSuccess
F9 B5,123
F9 FT,10
F9 B4,3
